In [ ]:
#NOTES FOR PLOTTING
#sources for plotting
#https://seaborn.pydata.org/tutorial/categorical.html
#https://flowingdata.com/2019/03/06/women-men-timeuse/

#add expected delay based on length and CAT per bridge --> 
#delaytime per bridge is delaytime(length) * prob_breakingdown(Cat)
#chose different scenario's --> Chose bridges breaking down? 

#traffic data is in ADTT (the total volume of vehicle traffic in both directions on a section of road for a year divided by 365 days)
# Still, the AADT gives a very good indication about the importance of a road, and about the number of truck passages per day or per year, which can be translated to the number of tonnes transported via that road per year.
#dividing the AADT by the number of lanes provides insight into how busy each lane is, and where we can expect most congestion

#To DO FROM NOW
#1. resolve error in summing
#2. combine delay time with traffic data --> laura script
#3. find a way to visualise the characteristics of the 10 most vulnerable/critical bridges with plotly or see seaborn thing

In [2]:
#STEP 1: import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

%matplotlib inline
import matplotlib.pyplot as plt
from scipy import ndimage
import seaborn as sns

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='itjallingii', api_key='hiFqLEhfwSwdDlsj9lT8')

In [30]:
#STEP 2 - RUN THIS LINE OF CODE FOR LOADING IN THE DATA
# Create a list of all the LRP's of the bridges
df_all_bmms = pd.read_csv('BMMS_overview.csv', encoding='utf-8',sep = ";")
df_traffic = pd.read_csv('data_traffic_road_segments.csv')

road_roadnames = pd.read_csv('_roadnames_list.csv')
road_roadnames_list = list(road_roadnames.columns.values)

#create empty float colums for summing average delay time & delete prerome points from previous assignment
df_all_bmms["average_delay"] = np.nan

#rename some columns in the traffic data for panda operations
df_traffic = df_traffic.rename(columns={'Total Traffic': 'Total_Traffic','Economic Traffic': 'Economic_Traffic','End Chainage': 'End_Chainage', 'No Lanes': 'No_Lanes','Start Chainage': 'Start_Chainage','Link no': 'Link_No','Link Name':'Link_Name',"Link Length": "Link_Length","Road name":"Road_Name"})

In [31]:
#STEP 3 fix some of the missing data for segments (90 segments did not allign --> see report)
for row in range(len(df_traffic)):
    if row > 0: 
        if df_traffic.Road_Name.iloc[row-1,] == df_traffic.Road_Name.iloc[row,]:
            preceding_segment_end_chainage = df_traffic.End_Chainage.iloc[row-1,]
            if df_traffic.End_Chainage.iloc[row-1,] != df_traffic.Start_Chainage.iloc[row,]:
                df_traffic.Start_Chainage.iloc[row,] = preceding_segment_end_chainage

In [129]:
#CHECK THE DATA CLEANING SCRIPT FOR MISSING SEGMENTS --> no non alligned segments should be present
df_traffic["missing_segment"] = ""
for row in range(len(df_traffic)):
    if row > 0:
        if df_traffic.Road_Name.iloc[row-1,] == df_traffic.Road_Name.iloc[row,]:
            if df_traffic.End_Chainage.iloc[row-1,] != df_traffic.Start_Chainage.iloc[row,]:
                df_traffic.missing_segment.iloc[row,] = True
df_traffic[df_traffic["missing_segment"] == True]

,Unnamed: 0,Road_Name,Link_No,Link_Name,Start LRP,Start Offset,Start_Chainage,End LRP,End Offset,End_Chainage,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized Total,Non Motorized Total,Total Traffic,missing_segment


In [32]:
#STEP 4 - RUN THIS LINE OF CODE FOR LINKING THE ROAD DATA WITH THE TRAFFIC (PER SEGMENT DATA) USING THE LINK NUMBER (lINK_NO)
df_selected = df_all_bmms
df_selected["Road_segment_no"] =""
df_selected["Road_segment_label"] =""

#fasterscript
for row_number in range(len(df_selected)):
    chainage = df_selected.chainage.iloc[row_number,]
    road_number = df_selected.road.iloc[row_number,]
    
    all_segments_in_road_df = df_traffic[df_traffic["Road_Name"] == road_number ]
    
    for segments in range(len(all_segments_in_road_df)):
        segment_start_chainage = all_segments_in_road_df.Start_Chainage.iloc[segments,]
        segment_end_chainage = all_segments_in_road_df.End_Chainage.iloc[segments,]
        segment_no = all_segments_in_road_df.Link_No.iloc[segments,]
        segment_label = all_segments_in_road_df.Link_Name.iloc[segments,]
                    
        if chainage >= segment_start_chainage:
            if chainage < segment_end_chainage:       
                df_selected.Road_segment_no.loc[row_number,] = segment_no
                df_selected.Road_segment_label.loc[row_number,] = segment_label    

df_BMSS_traffic_link = df_selected

In [147]:
#CHECK HOW MUCH MISSING DATA IS THERE
#3305 bridges mis segment data to this point --> chosen for deleting time for time reasons
number_of_missing_bridges_without_segment_data = (df_BMSS_traffic_link['Road_segment_no'].values == '').sum()
Missing_segment_data_brigdes_df = df_BMSS_traffic_link[df_BMSS_traffic_link['Road_segment_no'] == '']

#The missing bridges are mostly on Z roads -> impact on system results should not be that high and there is also often no traffic data for those roads
list_of_road_with_missing_segment_data = Missing_segment_data_brigdes_df.road.unique()

#drop the missing segments
df_BMSS_traffic_link = df_BMSS_traffic_link[df_BMSS_traffic_link["Road_segment_no"] != '']

In [33]:
#STEP 5 Determine vulnerability of bridges based on their scenario likelihood --> SEE REPORT FOR EXPLANATION
scenario = {'Scenario': [1,2,3,4,5,6,7,8], 'A': [0,0,0,0,0,0,0.05,0.10], 
            'B': [0,0,0,0,0.05,0.05,0.10,0.20],'C':[0,0,0,0.10,0.10,0.20,0.20,0.40],'D':[0,0.05,0.10,0.20,0.20,0.40,0.40,0.80],
            'Scenario_likelihood': [0,0.40,0.30,0.10,0.075,0.050,0.05,0.025]}
scenario_df = pd.DataFrame(data=scenario)
scenario_df = scenario_df.drop(scenario_df.index[0])

scenario_df["A_overall"] = scenario_df.Scenario_likelihood *scenario_df.A
scenario_df["B_overall"] = scenario_df.Scenario_likelihood *scenario_df.B
scenario_df["C_overall"] = scenario_df.Scenario_likelihood *scenario_df.C
scenario_df["D_overall"] = scenario_df.Scenario_likelihood *scenario_df.D

sum_row = {col: scenario_df[col].sum() for col in scenario_df}
scenario_df_sum = pd.DataFrame(sum_row, index=["Total"])
scenario_df = scenario_df.append(scenario_df_sum)
#---------> A_overall total is vulnerability of bridge

In [34]:
##STEP 6 RUN THIS LINE OF CODE FOR CALCULATING THE VULNERABILITY OF EACH ROAD SEGMENT IN THE TRAFFIC DATA FILE
#add coumns to df_traffic for output of vulnerability data
df_traffic["Total_A"] = ""
df_traffic["Total_B"] = ""
df_traffic["Total_C"] = ""
df_traffic["Total_D"] = ""
df_traffic["segment_vulnerability"] = ""
df_traffic["segment_vulnerability_length"] = ""

#iterate through all segments in the traffic file to get the vulnerability per segment
for i in range(len(df_traffic)):
    segment_name = df_traffic.iloc[i]['Link_No']
    
    #make small dataframe to get all bridges on that segment
    df_segments = df_BMSS_traffic_link[df_BMSS_traffic_link["Road_segment_no"] == segment_name]
    Bridge_count = df_segments.groupby(["condition"]).count()
    
    #create series object to count the bridges
    Bridge_count = Bridge_count["road"]
    categories = Bridge_count.index.values.tolist()
    
    #count number of bridges per category and check if they are they are there
    if 'A' in categories:
        total_A_Bridges = Bridge_count['A']
    else:
        total_A_Bridges = 0
    if 'B' in categories:
        total_B_Bridges = Bridge_count['B']
    else:
        total_B_Bridges = 0
    if 'C' in categories:
        total_C_Bridges = Bridge_count['C']
    else:
        total_C_Bridges = 0
    if 'D' in categories:
        total_D_Bridges = Bridge_count['D']
    else:
        total_D_Bridges = 0 
    
    #determine segment vulnerability based on scenario likehood and number of bridges on that segment
    segment_vulnerability = total_A_Bridges * scenario_df.iloc[7]['A_overall'] + total_B_Bridges * scenario_df.iloc[7]['B_overall'] + total_C_Bridges * scenario_df.iloc[7]['C_overall'] + total_D_Bridges * scenario_df.iloc[7]['D_overall'] 
        
    #store total bridges in segment traffic file    
    df_traffic.iloc[i, df_traffic.columns.get_loc('Total_A')] = total_A_Bridges
    df_traffic.iloc[i, df_traffic.columns.get_loc('Total_B')] = total_B_Bridges
    df_traffic.iloc[i, df_traffic.columns.get_loc('Total_C')] = total_C_Bridges
    df_traffic.iloc[i, df_traffic.columns.get_loc('Total_D')] = total_D_Bridges
    
    #store vulnerability in segment traffic file    
    df_traffic.iloc[i, df_traffic.columns.get_loc('segment_vulnerability')] = segment_vulnerability

df_traffic

,Unnamed: 0,Road_Name,Link_No,Link_Name,Start LRP,Start Offset,Start_Chainage,End LRP,End Offset,End_Chainage,...,Cart,Motorized Total,Non Motorized Total,Total_Traffic,Total_A,Total_B,Total_C,Total_D,segment_vulnerability,segment_vulnerability_length
0,0,N1,N1-1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,...,0.000000,2877.000000,181.121179,3058.121179,0,0,0,0,0,
1,2,N1,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,...,0.000000,3504.229927,215.411370,3719.641297,1,0,0,0,0.005,
2,3,N1,N1-3,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,...,0.000000,2999.100736,234.858517,3233.959253,1,0,0,0,0.005,
3,5,N1,N1-4,Shimrail - Katchpur (Left)N2 (Left),LRPS,7181,7.181,LRP009,260,8.763,...,0.000000,2749.850826,286.698827,3036.549652,0,0,0,0,0,
4,7,N1,N1-5,Katchpur - Madanpur (Left)N105 (Left),LRP009,260,8.763,LRP012,439,11.936,...,0.000000,5130.547291,379.232158,5509.779450,4,2,1,0,0.1,
5,9,N1,N1-6,Madanpur - Langalband (Left)Z1061 (Left),LRP012,439,11.936,LRP013,3411,15.935,...,0.000000,5518.678694,407.921477,5926.600171,7,1,0,0,0.05125,
6,11,N1,N1-7,Langalband - Mograpara Chowrasta (Left)Z1089 (...,LRP013,3411,15.935,LRP013,7520,20.044,...,0.000000,5641.500000,417.000000,6058.500000,4,9,2,0,0.26125,
7,13,N1,N1-8,Mograpara(Int.with Z1089)-Meghna Bridge West E...,LRP013,7520,20.044,LRP022,1935,23.564,...,0.000000,3395.557923,46.284953,3441.842876,7,6,0,0,0.1325,
8,15,N1,N1-9,Meghna Bridge Satrt-Bhaberchar (Left) z1063 (L...,LRP022,1935,23.564,LRP031,162,30.936,...,0.000000,3622.250000,49.375000,3671.625000,15,1,1,0,0.13875,
9,17,N1,N1-10,Bhaberchar(Int.with Z1063)-Daudkandi Bridge (L...,LRP031,162,30.936,LRP033,3664,36.271,...,0.000000,3622.250000,49.375000,3671.625000,18,2,1,0,0.17,


In [ ]:
#Step 7 - normalize data
from sklearn import preprocessing

columns_to_normalize = ["segment_vulnerability","Economic_Traffic"]
df_copy = df_traffic

for i in columns_to_normalize:
    # Create x, where x the 'scores' column's values as floats
    x = df_copy[[i]].values.astype(float)

    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()

    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)

    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled)

In [ ]:
x = df_copy[["Economic_Traffic"]].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)

In [74]:
#STEP 9 - Save the dataframe to CSV's to use them for the plotting scripts
df_traffic.to_csv('Traffic_segment_data.csv', sep=',', encoding = 'UTF-8')
df_BMSS_traffic_link.to_csv('Bridge_data_with_link_to_traffic.csv', sep=',', encoding = 'UTF-8')